In [1]:
%cd ../..

E:\システムトレード入門\trade_system_git_workspace


In [2]:
import pandas as pd
import numpy as np
from pytz import timezone
import datetime
from pathlib import Path
from tqdm.notebook import tqdm

In [3]:
from get_stock_price import StockDatabase

In [4]:
from portfolio.trade_transformer import PortfolioTransformer, PortfolioRestrictorIdentity, FeeCalculatorFree
from portfolio.price_supply import StockDBPriceSupplier, CannotGetAllDataError

In [5]:
from utils import TradeSystemBaseError

In [6]:
from utils import get_next_workday_intraday_datetime
import py_workdays

In [7]:
from portfolio.trade_transformer import PortfolioState, DataSupplyUnit

In [8]:
jst = timezone("Asia/Tokyo")
start_datetime = jst.localize(datetime.datetime(2020,11,10,0,0,0))
end_datetime = jst.localize(datetime.datetime(2020,12,10,0,0,0))
freq_str = "5T"
window = np.arange(0,50)

### エピソードの長さを計算する 

In [9]:
episode_length = len(py_workdays.extract_workdays_intraday_jp_index(pd.date_range(start_datetime, get_next_workday_intraday_datetime(end_datetime, freq_str, 300), freq=freq_str, closed="left")))
episode_length

1560

### 利用するticker_namesのリストを取得 

In [10]:
#db_path = Path("db/sub_stock_db/nikkei_255_stock.db")
db_path = Path("db/sub_stock_db/nikkei_255_stock_v2.db")

ticker_codes_df = pd.read_csv(Path("portfolio/rl_base/nikkei225.csv"), header=0)  # 自分で作成
ticker_codes = ticker_codes_df["code"].values.astype(str).tolist()
ticker_codes

['1333',
 '1332',
 '1605',
 '1802',
 '1801',
 '1963',
 '1812',
 '1803',
 '1925',
 '1808',
 '1928',
 '1721',
 '2002',
 '2269',
 '2914',
 '2503',
 '2501',
 '2801',
 '2502',
 '2802',
 '2531',
 '2871',
 '2282',
 '3101',
 '3401',
 '3402',
 '3103',
 '3863',
 '3861',
 '6988',
 '4021',
 '4061',
 '4901',
 '4004',
 '4911',
 '4043',
 '4183',
 '4042',
 '3407',
 '4208',
 '4188',
 '3405',
 '4005',
 '4631',
 '4063',
 '4452',
 '4519',
 '4578',
 '4506',
 '4502',
 '4507',
 '4523',
 '4568',
 '5019',
 '5020',
 '5101',
 '5202',
 '5214',
 '5232',
 '5233',
 '5301',
 '5332',
 '5333',
 '5401',
 '5406',
 '5411',
 '5541',
 '3436',
 '5703',
 '5706',
 '5707',
 '5711',
 '5713',
 '5714',
 '5801',
 '5802',
 '5803',
 '5901',
 '5631',
 '6103',
 '6113',
 '6301',
 '6302',
 '6305',
 '6326',
 '6361',
 '6367',
 '6471',
 '6472',
 '6473',
 '7004',
 '7011',
 '7013',
 '3105',
 '6479',
 '6501',
 '6503',
 '6504',
 '6506',
 '6645',
 '6674',
 '6701',
 '6702',
 '6703',
 '6724',
 '6752',
 '6758',
 '6762',
 '6770',
 '6841',
 '6857',
 

In [11]:
len(ticker_codes)

219

### デバッグ用の設定 

In [12]:
DataSupplyUnit.length_check = True
DataSupplyUnit.nan_check = True

PortfolioState.length_check = True
PortfolioState.nan_check = True
PortfolioState.portfoio_check = True

In [13]:
# stock_db
stock_db = StockDatabase(db_path)

# PriceSupplierの設定
price_supplier = StockDBPriceSupplier(stock_db,
                                      [],  # 最初は何の銘柄コードも指定しない
                                      None,
                                      freq_str,
                                      interpolate=True
                                     )

portfolio_transformer = PortfolioTransformer(price_supplier,
                                            portfolio_restrictor=PortfolioRestrictorIdentity(),
                                            use_ohlc="Close",
                                            initial_all_assets=1e6,  # 学習には関係ない
                                            fee_calculator=FeeCalculatorFree()
                                            )

price_supplier.episode_length = episode_length
price_supplier.ticker_names = [ticker_codes[0]]

portfolio_transformer.reset(start_datetime, window)
_,done = portfolio_transformer.step(np.array([1,0]))

In [16]:
# stock_db
stock_db = StockDatabase(db_path)

# PriceSupplierの設定
price_supplier = StockDBPriceSupplier(stock_db,
                                      [],  # 最初は何の銘柄コードも指定しない
                                      None,
                                      freq_str,
                                      interpolate=True
                                     )

portfolio_transformer = PortfolioTransformer(price_supplier,
                                            portfolio_restrictor=PortfolioRestrictorIdentity(),
                                            use_ohlc="Close",
                                            initial_all_assets=1e6,  # 学習には関係ない
                                            fee_calculator=FeeCalculatorFree()
                                            )

price_supplier.episode_length = 10000
price_supplier.ticker_names = ["1333"]

portfolio_transformer.reset(start_datetime, window)
_,done = portfolio_transformer.step(np.array([1,0]))

CannotGetAllDataError: Interpolate exceeds 10 % about tickers=['1333'], datetimes[2020-11-09 15:00:00+09:00,2021-07-14 11:25:00+09:00]

In [ ]:
bool_list = []

for ticker_code in tqdm(ticker_codes):
    price_supplier.ticker_names = list([ticker_code])
    try:
        price_supplier.episode_length = episode_length
        portfolio_transformer.reset(start_datetime, window)
        while True:
            _,done = portfolio_transformer.step(np.array([1,0]))
            if done:
                break
            
        bool_list.append(True)
    except TradeSystemBaseError:
        bool_list.append(False)

In [ ]:
bool_array = np.array(bool_list)
bool_array

In [ ]:
save_df = ticker_codes_df.loc[bool_array,:]
#save_df.to_csv(Path("portfolio/rl_base/nikkei225_modified.csv"), index=False, header=True)
save_df.to_csv(Path("portfolio/rl_base/nikkei225_modified_v2.csv"), index=False, header=True)